# Imports

In [ ]:
import numpy as np
import pandas as pd
import os 

import matplotlib.pyplot as plt
from matplotlib import cm

from mpl_toolkits.mplot3d import Axes3D

from sklearn.cluster import MeanShift,OPTICS,DBSCAN
from sklearn.cluster import  estimate_bandwidth

from tifffile import tifffile

import pyvista

# Data import

Read a folder with tiif files representing core cells

In [ ]:
dirname = './dbscan_nuclei/AVECNOIR/tif'
nuclei_img = []
for fname in os.listdir(dirname):
    im = tifffile.imread(os.path.join(dirname, fname))
    imarray = np.array(im)
    nuclei_img.append(imarray)

nuclei_img = np.asarray(nuclei_img)

# Analyse

In [ ]:
nuclei_img.shape

In [ ]:
for nucleus in nuclei_img :
    print(nucleus.shape)

In [ ]:
# Red pixel value of the first image of the first stack of the folder
nuclei_img[0][0][0][0][0]

# Data Preparation

make dataframe with all the pixels of each tiff image of the folder

In [ ]:
nuclei = pd.concat([(pd.DataFrame([[x,y,layer,nucleus_id,nucleus_img[layer][y][x]]], columns=['X','Y','Z','group','value'])) for nucleus_id ,nucleus_img in enumerate(nuclei_img) for layer in range(nucleus_img.shape[0]) for y in range(nucleus_img.shape[1]) for x in range(nucleus_img.shape[2]) if nucleus_img[layer][y][x].all() ],ignore_index=True)


In [ ]:
nuclei["Xreal"] = nuclei["X"] * 0.35 
nuclei["Yreal"] = nuclei["Y"] * 0.35 
nuclei["Zreal"] = nuclei["Z"]

In [ ]:
nuclei.sample(5)

In [ ]:
len(nuclei)

# Mean Shift

In [ ]:
# # MSC on 1st img

# myimg = nuclei.loc[nuclei['group'] == 0]
# pixels = myimg[["X","Y","Z"]].to_numpy()
# # bandwidth = estimate_bandwidth(pixels, quantile=0.125)
# # bandwidth = estimate_bandwidth(pixels, quantile=0.015, n_samples=int(len(pixels)*0.75))
# model = MeanShift( bin_seeding=True)
# model.fit(pixels)

# # Deduce variables from MSC
# cluster_centers = model.cluster_centers_
# labels = model.labels_
# cluster_label = np.unique(labels)
# n_clusters = len(cluster_centers)

# print("{} cluster found".format(n_clusters))

# #Ploting
# msc_fig = plt.figure(figsize=(150, 150))
# ax = msc_fig.add_subplot(111, projection ='3d')

# # plt.ion()

# cmap = plt.get_cmap('hsv')
# colors = cmap(np.linspace(0, 1.0, n_clusters))

# for kind, col in zip(cluster_label, colors):
#     my_members = [i for i in range(len(labels)) if labels[i] == kind]

#     plt.plot(pixels[my_members, 0], pixels[my_members, 1], pixels[my_members, 2],  marker='o', color=col)

# plt.show()

In [ ]:
# for img in range(len(nuclei_img)):
#     myimg = nuclei.loc[nuclei['group'] == img]
#     pixels = myimg[["X","Y","Z"]].to_numpy()
#     bandwidth = estimate_bandwidth(pixels, quantile=0.5)
#     # bandwidth = estimate_bandwidth(pixels, quantile=0.015, n_samples=int(len(pixels)*0.75))
#     model = MeanShift(bandwidth=bandwidth, bin_seeding=True)
#     model.fit(pixels)

#     # Deduce variables from MSC
#     cluster_centers = model.cluster_centers_
#     labels = model.labels_
#     cluster_label = np.unique(labels)
#     n_clusters = len(cluster_centers)

#     print("{} cluster found".format(n_clusters))

#     #Ploting
#     msc_fig = plt.figure(figsize=(150, 150))
#     ax = msc_fig.add_subplot(111, projection ='3d')

#     # plt.ion()

#     cmap = plt.get_cmap('hsv')
#     colors = cmap(np.linspace(0, 1.0, n_clusters))

#     for kind, col in zip(cluster_label, colors):
#         my_members = [i for i in range(len(labels)) if labels[i] == kind]
    
#         plt.plot(pixels[my_members, 0], pixels[my_members, 1], pixels[my_members, 2],  marker='o', color=col)

#     plt.show()

# Mesh Rendering

In [ ]:
plot_kwargs = { 'return_viewer': True, 'background': 'white'}

In [ ]:
cloudofPoint = nuclei[["Xreal","Yreal","Zreal"]].to_numpy()
# points is a 3D numpy array (n_points, 3) coordinates of a sphere
cloud = pyvista.PolyData(cloudofPoint)
# cloud.plot()
volume = cloud.delaunay_3d(alpha=.75)
shell = volume.extract_geometry()
shell.plot(plot_kwargs,jupyter_backend='ipygany')
shell.save("./Meshes/mesh%d.stl" % (id))


In [ ]:
shell.save("./Meshes/mesh%d.stl" % (id))

In [ ]:

id=0
mycell = nuclei.loc[nuclei['group'] == id]
coordonnees = mycell[["Xreal","Yreal","Zreal"]].to_numpy()

In [ ]:

# points is a 3D numpy array (n_points, 3) coordinates of a sphere
cloud = pyvista.PolyData(coordonnees)
# cloud.plot()
volume = cloud.delaunay_3d(alpha=.8)
shell = volume.extract_geometry()
shell.plot(plot_kwargs,jupyter_backend='ipygany')
# shell.save("./Meshes/mesh%d.stl" % (id))

In [ ]:
plot_kwargs = { 'return_viewer': True, 'background': 'white'}

for id in range (nuclei_img.shape[0]):
    mycell = nuclei.loc[nuclei['group'] == id]
    coordonnees = mycell[["Xreal","Yreal","Zreal"]].to_numpy()
    # points is a 3D numpy array (n_points, 3) coordinates of a sphere
    cloud = pyvista.PolyData(coordonnees)
    cloud.plot()
    volume = cloud.delaunay_3d(alpha=1.)
    shell = volume.extract_geometry()
    # shell.plot(plot_kwargs,jupyter_backend='ipygany')
    shell.save("./Meshes/mesh%d.stl" % (id))


In [ ]:
for elm in range (nuclei_img.shape[0]):
    mycell = nuclei.loc[nuclei['group'] == elm]
    coordonnees= mycell[["Xreal","Yreal","Zreal"]].to_numpy()
    
    # points is a 3D numpy array (n_points, 3) coordinates of a sphere
    cloud = pyvista.PolyData(coordonnees)
    volume = cloud.delaunay_3d(alpha=1.)
    newshell = volume.extract_geometry()
    print(elm, nuclei_img.shape[0])
    if elm == 0 :
        oldshell =newshell
    else :
        oldshell = oldshell.merge(newshell) 



plot_kwargs = { 'return_viewer': True, 'background': 'white'}
oldshell.plot(plot_kwargs,jupyter_backend='ipygany')